In [3]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
def run():
    use_cuda = torch.cuda.is_available()
    use_mps = torch.backends.mps.is_available()

    torch.manual_seed(1)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': 64}
    test_kwargs = {'batch_size': 1000}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('./data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('./data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    model.load_state_dict(torch.load("mnist_cnn.pt", map_location=device))
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, 14 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if True:
        torch.save(model.state_dict(), "mnist_cnn.pt")

In [11]:
run()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.000728
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.002307
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.010878
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.185279
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.033655
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.076386
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.004559
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.069402
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.012247
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.031004
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.069197
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.129260
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.001752
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.002414
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.016898
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.003730
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.052814
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.001726
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.072485
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.104714
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.006911
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.017295
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.112750
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.004712
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.000701
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.000487
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.001674
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.000324
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.007458
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.000487
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.000989
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.001428
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.030891
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.018505
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.022739
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.066919
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.000562
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.001140
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.000805
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.000107


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.005292
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.036937
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.013392
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.123946
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.013534
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.000090
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.029824
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.005274
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.010899
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.010015
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.024762
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.000308
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.065931
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.001094
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.000114
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.036037
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.006316
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.004497
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.028752
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.000146


Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.004975
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.007395
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.013875
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.000212
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.000712
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.027550
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.004252
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.035876
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.000613
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.097626
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.002585
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.002208
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.002488
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.003509
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.009845
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.001023
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.026362
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.000251
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.036784
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.015733



Test set: Average loss: 0.0330, Accuracy: 9920/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.008889
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.001899
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.006733
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.000463
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.000832
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.000912
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.003182
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.008717
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.005922
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.011286
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.006256
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.169831
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.007418
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.003912
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.012865
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.000213
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.066261
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.011956
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.003011
Train Epoch: 8 [12

Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.000442
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.003659
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.010212
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.005741
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.039659
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.002638
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.003933
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.006678
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.007235
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.034582
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.004942
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.004583
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.000193
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.000333
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.013469
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.107685
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.015024
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.000598
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.010158
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.002489


Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.014360
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.008177
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.026444
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.000349
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.005772
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.001868
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.043618
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.002527
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.002433
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.001275
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.012888
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.000267
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.002733
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.114680
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.002177
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.000932
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.017969
Train Epoch: 11 [37760/60000 (63%)]	Loss: 0.077700
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.005093
Train Epoch: 11 [39040/60000 (6

Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.006522
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.043200
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.000412
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.002851
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.009047
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.061397
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.000627
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.008455
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.078449
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.008638
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.005385
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.027357
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.000209
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.001873
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.009272
Train Epoch: 13 [19200/60000 (32%)]	Loss: 0.035262
Train Epoch: 13 [19840/60000 (33%)]	Loss: 0.025378
Train Epoch: 13 [20480/60000 (34%)]	Loss: 0.001483
Train Epoch: 13 [21120/60000 (35%)]	Loss: 0.014162
Train Epoch: 13 [21760/60000 (36

Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.009032
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.072344
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.009350
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.000033
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.016904
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.050737
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.001147
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.028979
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.000006
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.001246
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.001797

Test set: Average loss: 0.0309, Accuracy: 9927/10000 (99%)

